In [1]:
import pandas as pd
import csv
from copy import deepcopy

In [2]:
TOTAL_COST_SUM = 0 # For (temporary) A computation (obtained below)
TOTAL_NUM_FRAGMENTS = 0#243  # For (temporary) B computation

In [3]:
def to_array(s):
    s = s.replace(r'{', '')
    s = s.replace(r'}', '')
    s = s.replace(r' ', '')    

    if s == '': return []

    # return list(map(int,s.split(',')))

    return s.split(',')

In [4]:
original_df = pd.read_csv("Statement-Infomation.csv")
original_df

,Procedure,Statement,producer statement,consumer statement,cost,result size,Fragment
0,5,25,{},"{29, 30, 32}",0.061239,99090,"{88,95,96,97,101,102,103,87}"
1,5,26,{},"{29, 30, 32}",0.045698,73049,"{30,24}"
2,5,27,{},{29},0.047481,2880404,"{18,17,15,16,1,4}"
3,5,28,{},{30},0.064059,1441548,"{150,149,147,148,125,128}"
4,5,29,"{25,26,27}",{33},0.174254,188834,{}
5,5,30,"{25,26,28}",{33},0.169058,135789,{}
6,5,31,{},{32},0.063730,719384,"{186,211,210,208,209,190}"
7,5,32,"{25,26,31}",{33},0.171958,56139,{}
8,5,33,"{29,30,32}",{34},0.252135,380762,{}
9,5,34,{33},{35},1.182939,555294969,{}


In [5]:
original = {}

for i in range(len(original_df)):
    row = original_df.loc[i]

    # NOTE: Procedure is skipped in this algorithm
    tmp = []
    tmp.append(to_array(row[2])) 
    tmp.append(to_array(row[3]))
    tmp.append(row[4])
    tmp.append(row[5])

    fragments = to_array(row[6])
    tmp.append(fragments)
    
    original[str(row[1])] = tmp

    # d = dict([(row[1], tmp)])
    # original.update(d)

    TOTAL_COST_SUM += row[4]
    TOTAL_NUM_FRAGMENTS += len(fragments)

In [6]:
original['25']

[[],
 ['29', '30', '32'],
 0.061239,
 99090,
 ['88', '95', '96', '97', '101', '102', '103', '87']]

In [7]:
def rule_A(cost: int) -> bool:
    if cost > TOTAL_COST_SUM/4: return False    # type: ignore
    else: return True

def rule_B(fragments: list) -> bool:
    if len(fragments) > TOTAL_NUM_FRAGMENTS/4: return False
    else: return True

In [8]:
def compute_total_benefit(info: dict):
    total_benefit = 0
    for statement in info:
        row = info[statement]

        if len(row) > 5:
            total_benefit += row[6]
    
    return total_benefit

In [9]:
def update(info: dict, merged_keys: list[str], merge_key: str):
    '''
    Inplace function
    '''

    for merged_key in merged_keys:
        del info[merged_key]

    for statement in info:
        row = info[statement]

        def search_and_delete(array: list):
            deleted = False
            for i in reversed(range(len(array))):
                if array[i] in merged_keys:
                    del array[i]
                    deleted = True
                    
            if deleted: array.append(merge_key)
        
        search_and_delete(row[0])
        search_and_delete(row[1])

In [10]:
def merge(info: dict, producer_key: str, consumer_key: str, expected_cost: int, expected_fragments: list):
    '''
    Merge producer and consumer.
    '''
    p = info[producer_key]
    c = info[consumer_key]

    # Compute rest of merged info
    producer_statements = p[0] + c[0]
    producer_statements.remove(producer_key)    
    if producer_statements is None: producer_statements = []

    consumer_statements = p[1] + c[1]
    consumer_statements.remove(consumer_key)
    if consumer_statements is None: consumer_statements = []

    result_size = c[3]  #p[3] + c[3]

    consist = []
    if len(p) > 5:  # == p is merged statement
        consist.extend(p[5])
    else: consist.append(producer_key)
    
    if len(c) > 5: consist.extend(c[5])
    else: consist.append(consumer_key)

    consist = sorted(consist)

    benefit = p[3]

    # Pack the result
    merged = [
        producer_statements,
        consumer_statements,
        expected_cost,
        result_size,
        expected_fragments,
        consist,
        benefit
    ]


    # Make new key
    key = "EB-" + "-".join(map(str, list(consist)))

    return key, merged

In [11]:
def search(info: dict):
    # Sort candidate statements by expected benefits in descending order
    candidates_list = []
    for statement in info:
        row = info[statement]
        # NOTE: statement is consumer
        if len(row[1]) > 0: 
            candidates_list.append([statement, row[3]])
    candidates_list = sorted(candidates_list, key=lambda x: -x[1])

    '''
    Inner loop 
        for merging target statement with current largest expected benefit.

        exhuasted_inner is False
        when target statement was failed to merge (e.g. not passing rules).
    
    Outer loop 
        for when target statement was failed to merge
        proceed on statement with next largest expected benefit.

        exhuasted_outer is False 
        when entire loops are terminated without merging any.
    '''

    exhuasted_outer = False
    for candidate in candidates_list:
        producer_statement = candidate[0]
        target_row = info[producer_statement]
        
        exhuasted_inner = False
        updated_info = None
        
        for consumer_statement in target_row[1]: 
            consumer_row = info[consumer_statement]
            
            expected_cost = target_row[2] + consumer_row[2]
            expected_fragments = target_row[4] + consumer_row[4]

            # Validate
            if not(rule_A(expected_cost) and rule_B(expected_fragments)): continue
            

            # Merge
            merge_statement, merged_row = merge(info, producer_statement, consumer_statement, expected_cost, expected_fragments)


            # Update
            updated_info = deepcopy(info)

            updated_info[merge_statement] = (merged_row)
            update(updated_info, [producer_statement, consumer_statement], merge_statement)

            exhuasted_inner = True
            break
        
        
        if exhuasted_inner: 
            exhuasted_outer = True            
            break
    
    if exhuasted_outer: 
        search(updated_info) # type:ignore
    else: 
        print(info)
        return info

result = search(original)

{'25': [[], ['EB-28-30', 'EB-31-32', 'EB-27-29-33'], 0.061239, 99090, ['88', '95', '96', '97', '101', '102', '103', '87']], '26': [[], ['EB-28-30', 'EB-31-32', 'EB-27-29-33'], 0.045698, 73049, ['30', '24']], '34': [['EB-27-29-33'], ['35'], 1.182939, 555294969, []], '35': [['34'], [], 0.407706, 1, []], 'EB-28-30': [['25', '26'], ['EB-27-29-33'], 0.23311700000000002, 135789, ['150', '149', '147', '148', '125', '128'], ['28', '30'], 1441548], 'EB-31-32': [['25', '26'], ['EB-27-29-33'], 0.235688, 56139, ['186', '211', '210', '208', '209', '190'], ['31', '32'], 719384], 'EB-27-29-33': [['25', '26', 'EB-28-30', 'EB-31-32'], ['34'], 0.47387, 380762, ['18', '17', '15', '16', '1', '4'], ['27', '29', '33'], 188834]}


In [12]:
result

In [13]:
{
    '25': [[], ['EB-28-30', 'EB-31-32', 'EB-27-29-33'], 0.061239, 99090, ['88', '95', '96', '97', '101', '102', '103', '87']], 
    '26': [[], ['EB-28-30', 'EB-31-32', 'EB-27-29-33'], 0.045698, 73049, ['30', '24']], 
    '34': [['EB-27-29-33'], ['35'], 1.182939, 555294969, []], 
    '35': [['34'], [], 0.407706, 1, []], 
    'EB-28-30': [['25', '26'], ['EB-27-29-33'], 0.23311700000000002, 135789, ['150', '149', '147', '148', '125', '128'], ['28', '30'], 1441548], 
    'EB-31-32': [['25', '26'], ['EB-27-29-33'], 0.235688, 56139, ['186', '211', '210', '208', '209', '190'], ['31', '32'], 719384], 
    'EB-27-29-33': [['25', '26', 'EB-28-30', 'EB-31-32'], ['34'], 0.47387, 380762, ['18', '17', '15', '16', '1', '4'], ['27', '29', '33'], 188834]
}

{'25': [[],
  ['EB-28-30', 'EB-31-32', 'EB-27-29-33'],
  0.061239,
  99090,
  ['88', '95', '96', '97', '101', '102', '103', '87']],
 '26': [[],
  ['EB-28-30', 'EB-31-32', 'EB-27-29-33'],
  0.045698,
  73049,
  ['30', '24']],
 '34': [['EB-27-29-33'], ['35'], 1.182939, 555294969, []],
 '35': [['34'], [], 0.407706, 1, []],
 'EB-28-30': [['25', '26'],
  ['EB-27-29-33'],
  0.23311700000000002,
  135789,
  ['150', '149', '147', '148', '125', '128'],
  ['28', '30'],
  1441548],
 'EB-31-32': [['25', '26'],
  ['EB-27-29-33'],
  0.235688,
  56139,
  ['186', '211', '210', '208', '209', '190'],
  ['31', '32'],
  719384],
 'EB-27-29-33': [['25', '26', 'EB-28-30', 'EB-31-32'],
  ['34'],
  0.47387,
  380762,
  ['18', '17', '15', '16', '1', '4'],
  ['27', '29', '33'],
  188834]}